In [1]:
import numpy as np
import tensorflow as tf
import scipy.io
import math
import matplotlib.pyplot as plt
import statistics


In [2]:
def Krum(gradients, totalWorkers, malicious,m_list):
    with tf.name_scope("GAR_krum_tf"):
        nbworkers = totalWorkers
        nbbyzwrks=malicious
        nbselected=totalWorkers-malicious-1
        if nbselected<1:
            return float("NaN")
        assert len(gradients) > 0, "Empty list of gradient to aggregate"
      # Distance computations
        distances = []        
        for i in range(nbworkers - 1):
            dists = list()
            for j in range(i + 1, nbworkers):
                sqr_dst = tf.reduce_sum(tf.math.squared_difference(gradients[i], gradients[j]))
                dists.append(tf.math.negative(tf.where(tf.math.is_finite(sqr_dst), sqr_dst, tf.constant(np.inf, dtype=sqr_dst.dtype)))) 
                # Use of 'negative' to get the smallest distances and score indexes in 'nn.top_k
            distances.append(dists)
        scores = []
        for i in range(nbworkers):
            dists = []
            for j in range(nbworkers):
                if j == i:
                    continue
                if j < i:
                    dists.append(distances[j][i - j - 1])
                else:
                    dists.append(distances[i][j - i - 1])
            dists = tf.stack(values=dists)
            dists, _ = tf.nn.top_k(dists, k=(nbworkers - nbbyzwrks - 2), sorted=False)
            scores.append(tf.reduce_sum(dists))

        gradients = tf.stack(values=gradients)
        scores = tf.stack(values=scores)

        _, indexes = tf.nn.top_k(scores, k=nbselected, sorted=False)
        scores2=tf.math.negative(scores)
        
        if malicious>0:
            _,indexes2=tf.nn.top_k(scores2, k=malicious, sorted=False)
            m_list.append(tf.reduce_mean(tf.gather(gradients, indexes2), axis=0))

        return tf.reduce_mean(tf.gather(gradients, indexes), axis=0)


In [3]:

mat = scipy.io.loadmat('v_point_malicious_20_malicious_user_percent.mat')
mal_per=20
surface= scipy.io.loadmat('noise_surface_data.mat')
dic=mat['v_point_malicious']

X=surface['X']
Y=surface['Y']
X_coordinates=X[0][:151]
Y_coordinates=Y[:,0][:105]
[length, width]= [X_coordinates.size,Y_coordinates.size]
time_max=30

In [4]:
noise_map=[[[float("NaN") for x in range(width)] for y in range(length)] for z in range(time_max)]
malicious_map=[[[float("NaN") for x in range(width)] for y in range(length)] for z in range(time_max)]
for ti in range(time_max):
    for x in range(length):
        for y in range(width):
            num_workers=len(dic[x][y][ti])
            num_rounded = round(num_workers/10)*10
            num_malicious=int(num_rounded*0.01*mal_per)
            listCreated=[] #used to update mean of malicious user at x,y coordinate
            noise_map[ti][x][y]=Krum(dic[x][y][ti],num_workers,num_malicious,listCreated)
            if num_malicious>0:
                malicious_map[ti][x][y]=listCreated[0]

In [7]:
dict2={'noise_map_': noise_map,'malicious_points':malicious_map}
dict2.keys()

dict_keys(['noise_map_', 'malicious_points'])

In [8]:
scipy.io.savemat("noise_map_20_malicious_all_time_instances_using_partial_maps.mat", dict2 )